# Генеративно-состязательная сеть. Первые шаги в генерации картинок.

В этом тюториале мы получим основные графики, показанные на слайдах, а также натренируем Генеративно-состязательную модель на открытом наборе данных MNIST. Код написан с применением библиотеки PyTorch

Тетрадка является выдержкой из курса "Генеративные модели машинного обучения" https://github.com/HSE-LAMBDA/DeepGenerativeModels/, авторы Денис Деркач, Максим Артемьев, Артём Рыжиков. Ревью тетрадки: Михаил Гущин.

Задачи тюториала: 
1. Разобраться какая метрика лучше оценивает качество.
2. Получить представление о проблемах метрик. 
3. Натренировать простейшую генеративно-состязательную сеть (JSGAN), посмотреть на её качество.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline

import numpy as np


import torch
from torch import distributions as distrs
from torch.distributions.multivariate_normal import MultivariateNormal

from IPython.display import clear_output

Обозначим вспомогательные функции:

In [ ]:
# Красиво рисует двумерное распределение
def plot_2d_dots(dots, color='blue', label='None'):
    plt.ylim(-10, 10)
    plt.xlim(-10, 10)
    plt.scatter(dots[:, 0], dots[:, 1], s=1, c=color, label=label)

def create_distr(mu, sigma):
    return distrs.MultivariateNormal(mu, sigma)

# Оборачивает параметры распределения в торчевские тензоры
def get_parameters(mu=0., sigma=1.):
    train_mu = torch.Tensor([mu, mu])
    train_mu.requires_grad=True
    train_sigma = torch.Tensor([[sigma, 0.0],
                                [0.0, sigma]])
    train_sigma.requires_grad=True
    return train_mu, train_sigma

def sample(d, num):
    return d.sample(torch.Size([num]))

# Метрики качества

##### Зададим 2D Гаусс как целевое распределение

In [ ]:
mu = torch.Tensor([-5, -5])
sigma = torch.Tensor([[1., 0.0],
                      [0.0, 1.]])

target = create_distr(mu, sigma)
# x - samples from the target distribution
x = sample(target, 1000)
# px = p(x) = probability of target samples for the target distribution
px = target.log_prob(x).exp()

In [ ]:
plt.figure(figsize=(10, 10))
plot_2d_dots(x, color=px, label='target distr')
plt.legend()
plt.show()

### Аппроксимируем целевую функцию нашей, минимизируя KL дивергенцию

In [ ]:
# starting points
train_mu, train_sigma = get_parameters()

Q = create_distr(train_mu, train_sigma)
q_sample = sample(Q, 1000)
plt.figure(figsize=(10, 10))
plot_2d_dots(x, color='r', label='target distr')
plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')

plt.legend()
plt.show()

In [ ]:
def kl_loss(qx, px):
    # Clamp for the numerical stability 
    px, qx = px.clamp(min=1e-7), qx.clamp(min=1e-7)
    return torch.mean(px * (px.log() - qx.log())) # YOUR CODE

# train_mu and train_sigma are TRAINABLE parameters
train_mu, train_sigma = get_parameters()

In [ ]:
# Try replacing SGD with Adam
optim = torch.optim.SGD([train_mu, train_sigma], lr=0.1)

for i in range(5000):
    optim.zero_grad()
    Q = create_distr(train_mu, train_sigma)
    # qx = q(x) = probability of target samples for the train distribution
    qx = Q.log_prob(x).exp()
    loss = kl_loss(qx, px)
    loss.backward()
    optim.step()
    if i % 200 == 0:
        # plot pdfs
        clear_output(True)
        plt.figure(figsize=(10, 10))
        plt.title(f'KL={loss.item()}, iter={i}')
        plot_2d_dots(x, color='r', label='target distr')
        # q_sample - samples from the train distribution, just for visualization
        q_sample = sample(Q, 1000)
        plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')
        plt.legend()
        plt.show()

#### Вывод: всё довольно неплохо работает

### Попробуем с бимодальным распределением

In [ ]:
target1 = create_distr(torch.Tensor([-5, -5]), torch.Tensor([[1., 0.0], [0.0, 1.]]))
target2 = create_distr(torch.Tensor([4, 3]), torch.Tensor([[1., 0.0], [0.0, 1.]]))

x = torch.cat([sample(target1, 1000), sample(target2, 1000)])

px = target1.log_prob(x).exp() + target2.log_prob(x).exp()

In [ ]:
plt.figure(figsize=(10, 10))
plot_2d_dots(x, color=px, label='target distr')
plt.legend()
plt.show()

In [ ]:
train_mu, train_sigma = get_parameters()

Q = create_distr(train_mu, train_sigma)
q_sample = sample(Q, 1000)
plt.figure(figsize=(10, 10))
plot_2d_dots(x, color='r', label='target distr')
plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')

plt.legend()
plt.show()

In [ ]:
optim = torch.optim.SGD([train_mu, train_sigma], lr=0.1)

for i in range(5000):
    optim.zero_grad()
    Q = create_distr(train_mu, train_sigma)
    qx = Q.log_prob(x).exp()
    loss = kl_loss(qx, px)
    loss.backward()
    optim.step()
    if i % 200 == 0:
        # plot pdfs
        clear_output(True)
        plt.figure(figsize=(10, 10))
        plt.title(f'KL={loss.item()}, iter={i}')
        plot_2d_dots(x, color='r', label='target distr')
        q_sample = sample(Q, 1000)
        plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')
        plt.legend()
        plt.show()

### Вывод: Распределение, которое получается минимизацией обратной KL дивергенции пытается покрыть оба пика

#### Использование дивергенции Йенсена-Шеннона

In [ ]:
def js_div(qx, px):
    return 0.5 * kl_loss(px, 0.5*px+0.5*qx) + 0.5 * kl_loss(qx, 0.5*px+0.5*qx) # YOUR CODE

In [ ]:
train_mu, train_sigma = get_parameters(1, 1)

In [ ]:
optim = torch.optim.SGD([train_mu, train_sigma], lr=0.1)

for i in range(5000):
    optim.zero_grad()
    Q = create_distr(train_mu, train_sigma)
    qx = Q.log_prob(x).exp()
    loss = js_div(qx, px)
    loss.backward()
    optim.step()
    if i % 200 == 0:
        # plot pdfs
        clear_output(True)
        plt.figure(figsize=(10, 10))
        plt.title(f'JS={loss.item()}, iter={i}')
        plot_2d_dots(x, color='r', label='target distr')
        q_sample = sample(Q, 1000)
        plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')
        plt.legend()
        plt.show()

### Вывод: Распределение, которое получается минимизацией дивергенции ЙШ пытвается покрыть одну моду, но знает о второй.

### Можно также посмотреть на другие метрики расстояния, например, MSE
*as in [here](https://www.arxiv-vanity.com/papers/1611.04076/)

In [ ]:
def LSE_loss(qx, px):
    return torch.nn.MSELoss()(qx, px) # YOUR CODE

In [ ]:
train_mu, train_sigma = get_parameters(1, 1)

In [ ]:
optim = torch.optim.SGD([train_mu, train_sigma], lr=0.5)

for i in range(20000):
    optim.zero_grad()
    Q = create_distr(train_mu, train_sigma)
    qx = Q.log_prob(x).exp()
    loss = LSE_loss(qx, px)
    loss.backward()
    optim.step()
    if i % 200 == 0:
        # plot pdfs
        clear_output(True)
        plt.figure(figsize=(10, 10))
        plt.title(f'LSE={loss.item()}, iter={i}')
        plot_2d_dots(x, color='r', label='target distr')
        q_sample = sample(Q, 1000)
        plot_2d_dots(q_sample, color= Q.log_prob(q_sample).exp().detach(), label='train distr')
        plt.legend()
        plt.show()

# Перейдём к тренировке простой Генеративно-состязательной сети

Начнём с подготовки. Мы используем comet_ml, позволяющий легко просматривать результаты. Больше информации можно найти в этом курсе https://stepik.org/course/60000/ 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from comet_ml import Experiment

from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.utils as v_utils
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
batch_size = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'

mnist_transforms = transforms.Compose([ # Compose combines a number of transforms into one operation
    transforms.ToTensor(), # PIL Image -> Tensor
    transforms.Normalize([0.5], [0.5]) # input = (input - 0.5) / 0.5 -> x \sim input \in [-1, 1]
])

In [ ]:
# We can use torchvision package to get MNIST dataset

data_path = "../data/"

train_dataset = datasets.MNIST(data_path,
                               train=True,
                               transform=mnist_transforms,
                               target_transform=None,
                               download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)


In [ ]:
img, label = next(iter(train_loader))
print(f'Label: {label[0]}')
plt.imshow(img[0, 0, :,:])
plt.show()

Наша цель -- построить ГСС из двух разных сетей: генератора и дискриминатора.

1. Генератор берёт шум из латентного пространства и выводит изображение (1x28x28). Цель состоит в том, чтобы "обмануть" Дискриминатор.
2. Дискриминатор берёт изображение (1x28x28) и возвращает вероятность того, что изображение является реальным. Цель состоит в том, чтобы отличить реальные изображения от сгенерированных.

Используя бинарную кросс-энтропию, мы минимизируем ЙШ расстояние между вещественным и "сгенерированным" распределением, сдвигая "сгенерированные" изображения ближе к вещественным. 

Оригинальная статья [here](https://www.arxiv-vanity.com/papers/1406.2661/)

![alt text](GAN.png "GAN")

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.upsample = nn.Upsample(
            scale_factor=2,
            mode='bilinear',
            align_corners=True
        )
        self.layers = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), # 16x7x7
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            self.upsample, # 16x14x14
            nn.Conv2d(16, 32, 5, padding=2), # 32x14x14
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(),
            self.upsample, # 32x28x28
            nn.Conv2d(32, 32, 5, padding=2), # 32x28x28
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, 3, padding=1), # 32x28x28
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(),
        )
        self.final_layers = nn.Sequential(
            nn.Conv2d(32, 1, 3, padding=1), # 1x28x28
            nn.Tanh(), # 1x28x28 \in [-1, 1]
        )
        
        
    def forward(self, x):
        x = x.view(x.size(0), 1, 7, 7)
        x = self.layers(x)
        return self.final_layers(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 16, 7, stride=2, padding=3), # 16x14x14
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.Conv2d(16, 32, 5, stride=2, padding=2), # 32x7x7
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, 3, padding=1), # 32x7x7
            nn.BatchNorm2d(32), 
            nn.LeakyReLU(),
            nn.Conv2d(32, 1, 3, padding=1), # 1x7x7
            nn.LeakyReLU()
        )
        self.final_layers = nn.Sequential(
            nn.Linear(1*7*7, 1)
        )
    def forward(self,x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        return self.final_layers(x)

In [ ]:
from torchsummary import summary

generator = Generator().to(device)
print(summary(generator, (7*7, )))

In [ ]:
discriminator = Discriminator().to(device)
print(summary(discriminator, (1, 28, 28)))


In [ ]:
experiment = Experiment(api_key="lODeHEtCf7XLaV6DJrOfugNcA",
                        project_name="yandex-school-gan-mnist", workspace="holybayes")

LR = 0.001

optimizer_G = torch.optim.Adam(generator.parameters(), lr=LR)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LR)

criterion = torch.nn.BCEWithLogitsLoss()

n_epochs = 10

def sample_noise(batch, dims, mean=0, std=0.1):
    z = nn.init.normal_(torch.zeros(batch, dims, device=device), mean=mean, std=std)
    return z

In [ ]:
sample_interval = 500

for epoch in tqdm(range(n_epochs), desc='Epoch loop'):
    for iter_ind, (imgs, _) in tqdm(enumerate(train_loader), desc='Train loop', leave=False):
        
        ones = torch.ones(imgs.size(0), 1, device=device)
        zeros = torch.zeros(imgs.size(0), 1, device=device)
        step = epoch * len(train_loader) + iter_ind
        
        # generator update
        optimizer_G.zero_grad()
        fake_imgs = generator(sample_noise(imgs.size(0), 7*7))
        loss_G = criterion(discriminator(fake_imgs), ones)
        loss_G.backward()
        optimizer_G.step()
        
        # discriminator update
        optimizer_D.zero_grad()
        fake_imgs = generator(sample_noise(imgs.size(0), 7*7))
        err_real = criterion(discriminator(imgs.to(device)), ones)
        err_fake = criterion(discriminator(fake_imgs), zeros)
        loss_D = err_real + err_fake
        loss_D.backward()
        optimizer_D.step()
        
        experiment.log_metrics({'Generator loss': loss_G.item(),
                                'Discriminator loss': loss_D.item()},
                                epoch = epoch,
                                step = step)        

        if step % sample_interval == 0:
            plt.figure(figsize = (10,10))

            plt.title(
                f"[Epoch {epoch}/{n_epochs}]" + \
                f"[Batch {step%len(train_loader)}/{len(train_loader)}]" + \
                f"[D loss: {loss_D.item()}] [G loss: {loss_G.item()}]"
            )
            
            experiment.log_image(make_grid(fake_imgs.data[:25]).cpu().detach().numpy()[0, :, :])

            plt.imshow(make_grid(fake_imgs.data[:25]).cpu().detach().numpy()[0, :, :])
            experiment.log_figure()
            plt.clf()
experiment.end()

# *
Some usefull [tricks](https://github.com/soumith/ganhacks/blob/master/README.md) for training GAN's

GAN -- одна из самых популярных генеративных моделей на текущий момент. Она характеризуется:
1. чёткими картинками (в противовес вариационным автокодировщикам);
2. высокой гибкостью (мы можем делать всё более сильную модель генератора).

При этом у неё есть и недостатки:
1. коллапс мод (свойство метрики);
2. трудности сходимости (особенно для классического GAN, например, затухающие градиенты);
3. неявная функция финального распределения (мы можем хорошо сэмплировать, но функцию выписать трудно);